## Download and Clean Dataset

Let's start by importing the <em>pandas</em> and the Numpy libraries.

In [1]:
import pandas as pd
import numpy as np

We will be using the dataset provided in the assignment

<strong>The dataset is about the compressive strength of different samples of concrete based on the volumes of the different ingredients that were used to make them. Ingredients include:</strong>

<strong>1. Cement</strong>

<strong>2. Blast Furnace Slag</strong>

<strong>3. Fly Ash</strong>

<strong>4. Water</strong>

<strong>5. Superplasticizer</strong>

<strong>6. Coarse Aggregate</strong>

<strong>7. Fine Aggregate</strong>

Let's read the dataset into a <em>pandas</em> dataframe.

In [2]:
concrete_data = pd.read_csv('concrete_data.csv')
concrete_data.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


So the first concrete sample has 540 cubic meter of cement, 0 cubic meter of blast furnace slag, 0 cubic meter of fly ash, 162 cubic meter of water, 2.5 cubic meter of superplaticizer, 1040 cubic meter of coarse aggregate, 676 cubic meter of fine aggregate. Such a concrete mix which is 28 days old, has a compressive strength of 79.99 MPa. 

#### Let's check how many data points we have.

In [3]:
concrete_data.shape

(1030, 9)

So, there are approximately 1000 samples to train our model on. Because of the few samples, we have to be careful not to overfit the training data.

Let's check the dataset for any missing values.

In [4]:
concrete_data.describe()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
count,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000
mean,281.167864,73.895825,54.188350,181.567282,6.204660,972.918932,773.580485,45.662136,35.817961
std,104.506364,86.279342,63.997004,21.354219,5.973841,77.753954,80.175980,63.169912,16.705742
min,102.000000,0.000000,0.000000,121.800000,0.000000,801.000000,594.000000,1.000000,2.330000
25%,192.375000,0.000000,0.000000,164.900000,0.000000,932.000000,730.950000,7.000000,23.710000
50%,272.900000,22.000000,0.000000,185.000000,6.400000,968.000000,779.500000,28.000000,34.445000
75%,350.000000,142.950000,118.300000,192.000000,10.200000,1029.400000,824.000000,56.000000,46.135000
max,540.000000,359.400000,200.100000,247.000000,32.200000,1145.000000,992.600000,365.000000,82.600000


In [5]:
concrete_data.isnull().sum()

Cement                0
Blast Furnace Slag    0
Fly Ash               0
Water                 0
Superplasticizer      0
Coarse Aggregate      0
Fine Aggregate        0
Age                   0
Strength              0
dtype: int64

The data looks very clean and is ready to be used to build our model.

#### Split data into predictors and target

The target variable in this problem is the concrete sample strength. Therefore, our predictors will be all the other columns.

In [6]:
concrete_data_columns = concrete_data.columns
predictors = concrete_data[concrete_data_columns[concrete_data_columns != 'Strength']] # all columns except Strength
target = concrete_data['Strength'] # Strength column

Let's do a quick sanity check of the predictors and the target dataframes.

In [7]:
predictors.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360


In [8]:
target.head()

0    79.99
1    61.89
2    40.27
3    41.05
4    44.30
Name: Strength, dtype: float64

Finally, the last step is to normalize the data by substracting the mean and dividing by the standard deviation.

In [9]:
predictors_norm = (predictors - predictors.mean()) / predictors.std()
predictors_norm.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,0.862735,-1.217079,-0.279597
1,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,1.055651,-1.217079,-0.279597
2,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,3.551340
3,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,5.055221
4,-0.790075,0.678079,-0.846733,0.488555,-1.038638,0.070492,0.647569,4.976069


In [10]:
n_cols = predictors_norm.shape[1] # number of predictors

<a id="item1"></a>

<a id="item1"></a>

## Import Keras

#### Let's go ahead and import the Keras library

In [11]:
from tensorflow import keras # switch to tf.keras

As you can see, the TensorFlow backend was used to install the Keras library.

Let's import the rest of the packages from the Keras library that we will need to build our regressoin model.

In [12]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [13]:
# define regression model
def regression_model():
    # create model
    model = Sequential()
    model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(1))
    
    # compile model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

The above function creates a model that has three hidden layers each with 10 neurons and ReLU activation function. It uses the adam optimizer and the mean squared error as the loss function.

Let's import scikit-learn in order to randomly split the data into a training and test sets

In [14]:
from sklearn.model_selection import train_test_split

Splitting the data into a training and test sets by holding 30% of the data for testing

In [15]:
X_train, X_test, y_train, y_test = train_test_split(predictors_norm, target, test_size=0.3, random_state=42)

## Train and Test the Network

Let's call the function now to create our model.

In [16]:
# build the model
model = regression_model()

Next, we will train the model for 50 epochs.


In [17]:
# fit the model
epochs = 50
model.fit(X_train, y_train, epochs=epochs, verbose=2)

Epoch 1/50
23/23 - 0s - loss: 1600.9521
Epoch 2/50
23/23 - 0s - loss: 1589.1119
Epoch 3/50
23/23 - 0s - loss: 1579.8275
Epoch 4/50
23/23 - 0s - loss: 1568.2405
Epoch 5/50
23/23 - 0s - loss: 1549.4855
Epoch 6/50
23/23 - 0s - loss: 1517.7579
Epoch 7/50
23/23 - 0s - loss: 1465.7991
Epoch 8/50
23/23 - 0s - loss: 1385.1570
Epoch 9/50
23/23 - 0s - loss: 1266.9918
Epoch 10/50
23/23 - 0s - loss: 1108.0020
Epoch 11/50
23/23 - 0s - loss: 909.7438
Epoch 12/50
23/23 - 0s - loss: 696.6431
Epoch 13/50
23/23 - 0s - loss: 503.3608
Epoch 14/50
23/23 - 0s - loss: 374.6065
Epoch 15/50
23/23 - 0s - loss: 303.9287
Epoch 16/50
23/23 - 0s - loss: 271.8271
Epoch 17/50
23/23 - 0s - loss: 254.4139
Epoch 18/50
23/23 - 0s - loss: 239.8136
Epoch 19/50
23/23 - 0s - loss: 228.0815
Epoch 20/50
23/23 - 0s - loss: 217.2532
Epoch 21/50
23/23 - 0s - loss: 207.8567
Epoch 22/50
23/23 - 0s - loss: 200.3302
Epoch 23/50
23/23 - 0s - loss: 193.2151
Epoch 24/50
23/23 - 0s - loss: 187.5520
Epoch 25/50
23/23 - 0s - loss: 183.0149

Next we need to evaluate the model on the test data.

In [18]:
loss_val = model.evaluate(X_test, y_test)
y_pred = model.predict(X_test)
loss_val

10/10 [==============================] - 0s 575us/step - loss: 135.2031


135.20306396484375

Now we need to compute the mean squared error between the predicted concrete strength and the actual concrete strength.

Let's import the mean_squared_error function from Scikit-learn.

In [19]:
from sklearn.metrics import mean_squared_error

In [20]:
mean_square_error = mean_squared_error(y_test, y_pred)
mean = np.mean(mean_square_error)
standard_deviation = np.std(mean_square_error)
print(mean, standard_deviation)

135.20304308131006 0.0


Create a list of 50 mean squared errors and report mean and the standard deviation of the mean squared errors.

In [21]:
total_mean_squared_errors = 50
epochs = 50
mean_squared_errors = []
for i in range(0, total_mean_squared_errors):
    X_train, X_test, y_train, y_test = train_test_split(predictors_norm, target, test_size=0.3, random_state=i)
    model.fit(X_train, y_train, epochs=epochs, verbose=0)
    MSE = model.evaluate(X_test, y_test, verbose=0)
    print("MSE "+str(i+1)+": "+str(MSE))
    y_pred = model.predict(X_test)
    mean_square_error = mean_squared_error(y_test, y_pred)
    mean_squared_errors.append(mean_square_error)

mean_squared_errors = np.array(mean_squared_errors)
mean = np.mean(mean_squared_errors)
standard_deviation = np.std(mean_squared_errors)

print('\n')
print("Below is the mean and standard deviation of " +str(total_mean_squared_errors) + " mean squared errors with normalized data. Total number of epochs for each training is: " +str(epochs) + "\n")
print("Mean: "+str(mean))
print("Standard Deviation: "+str(standard_deviation))

MSE 1: 103.78202056884766
MSE 2: 109.97171020507812
MSE 3: 74.65216064453125
MSE 4: 66.07154083251953
MSE 5: 57.000797271728516
MSE 6: 54.07426834106445
MSE 7: 52.31381607055664
MSE 8: 39.077415466308594
MSE 9: 35.636714935302734
MSE 10: 34.75309753417969
MSE 11: 33.9312629699707
MSE 12: 27.01409912109375
MSE 13: 34.4276123046875
MSE 14: 35.707313537597656
MSE 15: 27.221006393432617
MSE 16: 22.436084747314453
MSE 17: 26.753150939941406
MSE 18: 28.44582176208496
MSE 19: 22.499544143676758
MSE 20: 30.17768669128418
MSE 21: 23.584688186645508
MSE 22: 24.222524642944336
MSE 23: 20.85762596130371
MSE 24: 24.99100685119629
MSE 25: 23.210458755493164
MSE 26: 27.490705490112305
MSE 27: 22.539003372192383
MSE 28: 20.27504539489746
MSE 29: 26.033483505249023
MSE 30: 22.19057273864746
MSE 31: 22.77389907836914
MSE 32: 22.942665100097656
MSE 33: 20.92616081237793
MSE 34: 23.10943031311035
MSE 35: 26.992643356323242
MSE 36: 28.91161346435547
MSE 37: 20.979352951049805
MSE 38: 22.052696228027344
MSE